In [1]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from xgboost import XGBRegressor

In [2]:
!pip install git+https://github.com/NextBrain-ai/nbsynthetic.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/NextBrain-ai/nbsynthetic.git to /tmp/pip-req-build-kvf_kym4
  Running command git clone --filter=blob:none --quiet https://github.com/NextBrain-ai/nbsynthetic.git /tmp/pip-req-build-kvf_kym4
  Resolved https://github.com/NextBrain-ai/nbsynthetic.git to commit 9241207e521ade47c54cff6fcf3d3d368a8efb01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 KB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.0/48.0 KB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for nbsynthetic: filename=nbsynthetic-0.2.0-py3-none-any.whl size=35899 sha256=15223e445450cf90cf4e4e72ff83e6859f337faac1662628ceed9414ddbb36a5
  Stored in directory: /tmp/pip-ep

In [3]:
from nbsynthetic.data import input_data
from nbsynthetic.data_preparation import SmartBrain
from nbsynthetic.vgan import GAN
from nbsynthetic.synthetic import synthetic_data

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
data1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Techson Hackathon (temp)/PS1/data/1A/PS1A_train.csv')
data2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Techson Hackathon (temp)/PS1/data/1B/PS1B_train.csv')
data3 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Techson Hackathon (temp)/PS1/data/1C/PS1C_train.csv')
data4 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Techson Hackathon (temp)/PS1/data/1D/PS1D_train.csv')

df1 = data1.drop('Index', axis=1)
df2 = data2.copy()
df3 = data3.drop('Index', axis=1)
df4 = data4.drop('Index', axis=1)


print('DF1 Shape: ', df1.shape)
print('DF2 Shape: ', df2.shape)
print('DF3 Shape: ', df3.shape)
print('DF4 Shape: ', df4.shape)
print()
print('DF1 Missing: ', df1.isnull().any().any())
print('DF2 Missing: ', df2.isnull().any().any())
print('DF3 Missing: ', df3.isnull().any().any())
print('DF4 Missing: ', df4.isnull().any().any())

DF1 Shape:  (6951, 10)
DF2 Shape:  (307, 9)
DF3 Shape:  (1000, 30)
DF4 Shape:  (835, 9)

DF1 Missing:  False
DF2 Missing:  False
DF3 Missing:  False
DF4 Missing:  False


In [6]:
def XGB_reg(data, label):
  Xt, Xv, yt, yv = train_test_split(data, label)
  model = XGBRegressor()
  scores = -cross_val_score(model, data, label,cv=10, scoring='neg_root_mean_squared_error')
  return model, np.mean(scores)

def XGB_class(data, label):
  Xt, Xv, yt, yv = train_test_split(data, label)
  model = XGBClassifier()
  scores = -cross_val_score(model, data, label,cv=10, scoring='neg_root_mean_squared_error')
  return model, np.mean(scores)

In [7]:
# XGB1, XGBScores1 = XGB_model(df1.drop('Target', axis=1), df1['Target'])
# XGBScores1

# DF1

In [8]:
df1.head()

,C1,C2,C3,C4,C5,C6,C7,C8,C9,Target
0,3,9,9,0,6,0,1,0.60,0.43,342
1,3,7,11,0,0,0,1,0.84,0.53,243
2,1,1,23,0,4,1,2,0.40,0.82,91
3,3,9,6,0,0,0,1,0.58,0.88,20
4,2,5,9,0,6,0,1,0.54,0.49,218


In [9]:
cat_cols1 = [col for col in df1.columns if df1[col].nunique() <= 5]
cat_cols1

['C1', 'C4', 'C6', 'C7']

In [10]:
Xt1, Xv1, _, _ = train_test_split(df1, df1['Target'], random_state = 0)

In [11]:
baseline1 = XGBRegressor()
baseline1.fit(Xt1.drop('Target', axis=1), Xt1['Target'])
print('Baseline RMSE: ', mean_squared_error(Xv1['Target'], baseline1.predict(Xv1.drop('Target', axis=1)), squared=False))

Baseline RMSE:  71.46855903669915


In [12]:
Xt1_temp = Xt1.copy()
Xt1_temp[cat_cols1] = Xt1_temp[cat_cols1].astype('category')

In [13]:
SB = SmartBrain() 
Xt1_temp = SB.nbEncode(Xt1_temp)
Xt1_fake = synthetic_data(GAN, Xt1_temp, samples = 5000)

  0%|          | 0/218 [00:00<?, ?it/s]

1/1 [==============================] - 0s 243ms/step


Epoch (1/10) | D. loss: 0.71 | G. loss: 0.68 |:   0%|          | 1/218 [00:05<20:50,  5.76s/it]

1/1 [==============================] - 0s 35ms/step


Epoch (1/10) | D. loss: 0.68 | G. loss: 0.67 |:   1%|          | 2/218 [00:05<08:56,  2.48s/it]

...

Epoch (10/10) | D. loss: 0.69 | G. loss: 0.70 |: 100%|#########9| 217/218 [01:09<00:00,  4.05it/s]

1/1 [==============================] - 0s 29ms/step


Epoch (10/10) | D. loss: 0.71 | G. loss: 0.69 |: 100%|##########| 218/218 [01:09<00:00,  3.14it/s]

  1/157 [..............................] - ETA: 5s

157/157 [==============================] - 1s 7ms/step


In [14]:
Xt1_fake.head()

,C1,C2,C3,C4,C5,C6,C7,C8,C9,Target
0,0,1.046671,9.980379,0,1.940551,1,2,0.504838,0.172873,738.729805
1,3,6.336568,2.559584,0,1.984528,0,0,0.032720,0.032615,24.359217
2,3,4.322175,4.491505,0,1.001351,1,0,0.263150,0.324155,231.159156
3,3,8.113480,0.132981,0,5.799260,0,0,0.021011,0.033733,2.526037
4,3,3.250739,12.543655,0,0.534439,1,0,0.589270,0.062702,678.013226


In [15]:
Xt1_final = pd.concat([Xt1, Xt1_fake], axis=0)
XGB1 = XGBRegressor()
XGB1.fit(Xt1_final.drop('Target', axis=1), Xt1_final['Target'])
print('RMSE: ', mean_squared_error(Xv1['Target'], XGB1.predict(Xv1.drop('Target', axis=1)), squared=False))

RMSE:  72.51827188148691


# DF2

In [16]:
df2.head()

,C1,C2,C3,C4,C5,C6,C7,C8,Outcome
0,5,187,76,27,207,43.6,1.034,53,1
1,3,129,64,29,115,26.4,0.219,28,1
2,0,131,0,0,0,43.2,0.270,26,1
3,10,115,0,0,0,0.0,0.261,30,1
4,0,107,60,25,0,26.4,0.133,23,0


In [17]:
cat_cols2 = [col for col in df2.columns if df2[col].nunique() <= 5]
cat_cols2

['Outcome']

In [18]:
Xt2, Xv2, _, _ = train_test_split(df2, df2['Outcome'], random_state = 0)

In [19]:
baseline2 = XGBClassifier()
baseline2.fit(Xt2.drop('Outcome', axis=1), Xt2['Outcome'])
print('Baseline Accuracy: ', accuracy_score(Xv2['Outcome'], baseline2.predict(Xv2.drop('Outcome', axis=1))))

Baseline Accuracy:  0.7662337662337663


In [20]:
Xt2_temp = Xt2.copy()
Xt2_temp[cat_cols2] = Xt2_temp[cat_cols2].astype('category')

In [21]:
SB = SmartBrain() 
Xt2_temp = SB.nbEncode(Xt2_temp)
Xt2_fake = synthetic_data(GAN, Xt2_temp, samples = 5000)

  0%|          | 0/10 [00:00<?, ?it/s]

1/1 [==============================] - 0s 249ms/step


Epoch (1/10) | D. loss: 0.69 | G. loss: 0.67 |:  10%|#         | 1/10 [00:08<01:13,  8.12s/it]

1/1 [==============================] - 0s 116ms/step


Epoch (1/10) | D. loss: 0.68 | G. loss: 0.68 |:  20%|##        | 2/10 [00:08<00:29,  3.71s/it]

...

1/1 [==============================] - 0s 31ms/step


Epoch (10/10) | D. loss: 0.65 | G. loss: 0.72 |:  90%|######### | 9/10 [00:03<00:00,  3.41it/s]

1/1 [==============================] - 0s 31ms/step


Epoch (10/10) | D. loss: 0.70 | G. loss: 0.71 |: 100%|##########| 10/10 [00:03<00:00,  2.78it/s]


157/157 [==============================] - 1s 5ms/step


In [22]:
Xt2_fake.head()

,C1,C2,C3,C4,C5,C6,C7,C8,Outcome
0,5.934783,197.000000,107.999534,15.065217,16.173913,6.669019,2.413241,66.999908,1
1,0.000000,0.000000,0.000000,98.726036,129.391304,0.000000,0.084000,21.000000,0
2,10.610895,0.000000,106.889049,98.950839,0.000000,59.400000,2.420000,21.000000,0
3,0.000000,196.982683,107.972522,99.000000,730.287120,39.532723,2.417308,25.000000,0
4,0.000000,197.000000,0.000000,0.000000,0.000000,0.000000,0.129314,66.999939,1


In [23]:
Xt2_final = pd.concat([Xt2, Xt2_fake], axis=0)
XGB2 = XGBClassifier()
XGB2.fit(Xt2_final.drop('Outcome', axis=1), Xt2_final['Outcome'])
print('Accuracy: ', accuracy_score(Xv2['Outcome'], XGB2.predict(Xv2.drop('Outcome', axis=1))))

Accuracy:  0.7922077922077922


# DF 3

In [24]:
df3.head()

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,...,C21,C22,C23,C24,C25,C26,C27,C28,C29,Target
0,-0.633509,-0.287744,0.222853,-2.998039,-0.690275,-0.674866,-0.404163,0.405966,-2.319937,0.775369,...,0.031281,0.395767,-0.097444,-0.357012,-0.230728,-0.239019,0.272303,0.128075,29.89,0
1,-1.432863,-1.290936,1.081047,1.689179,1.857100,-0.633250,-0.808269,0.466850,-0.099381,0.161682,...,0.233531,0.229839,0.367753,-0.345799,-0.614424,-0.322642,0.159902,0.221186,38.94,0
2,-1.275721,-0.939394,1.391180,-1.432628,-1.598677,0.829657,1.103927,-0.276995,0.600442,-0.757897,...,0.161042,0.918642,0.382627,-0.575711,0.240417,-0.217267,0.191545,-0.014056,399.40,0
3,2.117287,0.170442,-1.945039,0.273686,0.740743,-0.530378,0.199080,-0.220303,0.476406,-0.388658,...,-0.399412,-1.022962,0.261353,-0.051819,-0.165381,0.207375,-0.060959,-0.035671,1.79,0
4,2.196452,-1.362427,-0.280416,-1.437635,-1.381315,-0.249888,-1.454115,-0.136940,-0.081281,1.275863,...,-0.496381,-0.743399,0.391387,-0.437921,-0.595208,-0.528237,0.009113,-0.047769,39.95,0


In [25]:
cat_cols3 = [col for col in df3.columns if df3[col].nunique() <= 5]
cat_cols3

['Target']

In [26]:
Xt3, Xv3, _, _ = train_test_split(df3, df3['Target'], random_state = 0)

In [27]:
baseline3 = XGBClassifier()
baseline3.fit(Xt3.drop('Target', axis=1), Xt3['Target'])
print('Baseline Accuracy: ', accuracy_score(Xv3['Target'], baseline3.predict(Xv3.drop('Target', axis=1))))

Baseline Accuracy:  0.992


In [28]:
Xt3_temp = Xt3.copy()
Xt3_temp[cat_cols3] = Xt3_temp[cat_cols3].astype('category')

In [29]:
SB = SmartBrain() 
Xt3_temp = SB.nbEncode(Xt3_temp)
Xt3_fake = synthetic_data(GAN, Xt3_temp, samples = 5000)

  0%|          | 0/32 [00:00<?, ?it/s]

1/1 [==============================] - 0s 325ms/step


Epoch (1/10) | D. loss: 0.70 | G. loss: 0.72 |:   3%|3         | 1/32 [00:12<06:12, 12.02s/it]

1/1 [==============================] - 0s 62ms/step


Epoch (1/10) | D. loss: 0.70 | G. loss: 0.71 |:   6%|6         | 2/32 [00:12<02:35,  5.17s/it]

...

Epoch (10/10) | D. loss: 0.55 | G. loss: 0.90 |:  97%|#########6| 31/32 [00:07<00:00,  4.86it/s]

1/1 [==============================] - 0s 31ms/step


Epoch (10/10) | D. loss: 0.82 | G. loss: 0.94 |: 100%|##########| 32/32 [00:07<00:00,  4.27it/s]

 10/157 [>.............................] - ETA: 0s

157/157 [==============================] - 1s 7ms/step


In [30]:
Xt3_fake.head()

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,...,C21,C22,C23,C24,C25,C26,C27,C28,C29,Target
0,2.367338,-17.820657,-18.330609,11.150310,10.180816,-5.269282,-9.470890,7.111104,6.987922,-15.238382,...,-3.033864,-2.646735,-5.765102,-1.533815,-2.482740,-1.213238,-6.026794,-3.272030,214.024063,0
1,-21.472938,-15.766999,-2.171412,-3.779712,-14.156858,-5.302778,-14.906808,-10.506897,-7.170900,-13.764256,...,-5.140636,-2.647036,-5.767336,-1.848189,-0.753122,-0.646017,-6.028104,-3.269819,0.304197,0
2,-21.472949,-17.821552,-17.940459,-2.634212,-14.162258,-5.693790,-14.910983,-9.808163,-7.175066,-9.862937,...,7.414459,-2.625981,-5.426924,-1.848845,-2.470869,-0.505655,-6.028110,-2.840343,3363.030762,0
3,-21.440662,-17.826932,0.789337,10.029854,-14.163150,8.287778,-14.913330,7.273197,-6.750933,4.651272,...,-5.140678,-2.647964,2.507742,-1.848845,-1.647910,-1.149228,-6.028106,-3.273510,732.475098,0
4,-10.542310,-17.416792,-15.458548,-2.724495,-14.163073,-0.800787,-14.121962,-10.519831,-7.008150,-10.986882,...,-5.140334,-2.269053,-5.669400,-1.848825,-2.390614,1.633096,-5.910121,-3.266199,3491.559814,0


In [31]:
Xt3_final = pd.concat([Xt3, Xt3_fake], axis=0)
XGB3 = XGBClassifier()
XGB3.fit(Xt3_final.drop('Target', axis=1), Xt3_final['Target'])
print('Accuracy: ', accuracy_score(Xv3['Target'], XGB3.predict(Xv3.drop('Target', axis=1))))

Accuracy:  0.988


# DF 4

In [32]:
df4.head()

,C1,C2,C3,C4,C5,C6,C7,C8,Target
0,1,0.665,0.500,0.175,1.2975,0.6075,0.3140,0.3150,9
1,2,0.535,0.460,0.145,0.7875,0.3395,0.2005,0.2000,8
2,1,0.625,0.495,0.155,1.0485,0.4870,0.2120,0.3215,11
3,2,0.625,0.490,0.110,1.1360,0.5265,0.1915,0.2925,9
4,3,0.410,0.300,0.090,0.2800,0.1410,0.0575,0.0750,8


In [33]:
cat_cols4 = [col for col in df4.columns if df4[col].nunique() <= 5]
cat_cols4

['C1']

In [34]:
Xt4, Xv4, _, _ = train_test_split(df4, df4['Target'], random_state = 0)

In [35]:
baseline4 = XGBRegressor()
baseline4.fit(Xt4.drop('Target', axis=1), Xt4['Target'])
print('Baseline RMSE: ', mean_squared_error(Xv4['Target'], baseline4.predict(Xv4.drop('Target', axis=1)), squared=False))

Baseline RMSE:  2.5123891555079343


In [36]:
Xt4_temp = Xt4.copy()
Xt4_temp[cat_cols4] = Xt4_temp[cat_cols4].astype('category')

In [37]:
SB = SmartBrain() 
Xt4_temp = SB.nbEncode(Xt4_temp)
Xt4_fake = synthetic_data(GAN, Xt4_temp, samples = 5000)

  0%|          | 0/27 [00:00<?, ?it/s]

1/1 [==============================] - 0s 175ms/step


Epoch (1/10) | D. loss: 0.69 | G. loss: 0.69 |:   4%|3         | 1/27 [00:06<02:38,  6.09s/it]

1/1 [==============================] - 0s 47ms/step


Epoch (1/10) | D. loss: 0.69 | G. loss: 0.71 |:   7%|7         | 2/27 [00:06<01:05,  2.63s/it]

...

Epoch (10/10) | D. loss: 0.67 | G. loss: 0.90 |:  96%|#########6| 26/27 [00:05<00:00,  3.63it/s]

1/1 [==============================] - 0s 50ms/step


Epoch (10/10) | D. loss: 0.56 | G. loss: 0.84 |: 100%|##########| 27/27 [00:05<00:00,  4.66it/s]

  1/157 [..............................] - ETA: 6s

157/157 [==============================] - 1s 8ms/step


In [38]:
Xt4_fake.head()

,C1,C2,C3,C4,C5,C6,C7,C8,Target
0,0,0.140002,0.128668,0.015001,0.881978,0.006373,0.006536,0.193185,18.139315
1,2,0.769998,0.571077,0.208220,0.448239,1.232000,0.185239,0.161820,3.001065
2,2,0.222174,0.105000,0.187514,0.014500,0.159599,0.003500,0.092892,3.000000
3,0,0.770000,0.615000,0.234975,2.508454,1.232000,0.526000,0.447583,26.000000
4,0,0.140722,0.171522,0.015285,0.022605,0.061010,0.230674,0.287281,11.000000


In [39]:
Xt4_final = pd.concat([Xt4, Xt4_fake], axis=0)
XGB4 = XGBRegressor()
XGB4.fit(Xt4_final.drop('Target', axis=1), Xt4_final['Target'])
print('RMSE: ', mean_squared_error(Xv4['Target'], XGB4.predict(Xv4.drop('Target', axis=1)), squared=False))

RMSE:  2.4477632676655796


# Test data predictions

In [42]:
tdf1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Techson Hackathon (temp)/PS1/data/1A/PS1A_test.csv', index_col='Index')
tdf2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Techson Hackathon (temp)/PS1/data/1B/PS1B_test.csv', index_col='Index')
tdf3 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Techson Hackathon (temp)/PS1/data/1C/PS1C_test.csv', index_col='Index')
tdf4 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Techson Hackathon (temp)/PS1/data/1D/PS1D_test.csv', index_col='Index')

In [49]:
tpred1 = XGB1.predict(tdf1)
tpred2 = XGB2.predict(tdf2)
tpred3 = XGB3.predict(tdf3)
tpred4 = XGB4.predict(tdf4)

dfpred1 = pd.DataFrame({'Index':tdf1.index, 'Target': tpred1})
dfpred2 = pd.DataFrame({'Index':tdf2.index, 'Target': tpred2})
dfpred3 = pd.DataFrame({'Index':tdf3.index, 'Target': tpred3})
dfpred4 = pd.DataFrame({'Index':tdf4.index, 'Target': tpred4})

In [54]:
dfpred1.to_csv('PS1_A_SVSsy_baka.csv', index=False)
dfpred2.to_csv('PS1_B_SVSsy_baka.csv', index=False)
dfpred3.to_csv('PS1_C_SVSsy_baka.csv', index=False)
dfpred4.to_csv('PS1_D_SVSsy_baka.csv', index=False)